In [50]:
import numpy as np
import pandas as pd
from nltk.stem.porter import *

In [11]:
epr_identified = pd.read_csv('more_files/OBS_ReadingCategories_EPR_1.csv').iloc[1:63,2:]

In [59]:
ps = PorterStemmer()
all_categories_flattened = set([str(x).strip().lower() for y in epr_identified.values.tolist() for x in y if str(x) != 'nan'])
# for line in all_categories_flattened:
#     print([ps.stem(x) for x in line.split()])

all_categories_stemmed = {' '.join([ps.stem(x) for x in line.split()]) for line in all_categories_flattened}

In [60]:
df = pd.read_csv(
    'data_first_email/20190118_reading_specialists.csv'
)
obs_full = list(df.OBS)
obs_full = [sent.replace('word by word', 'word-by-word') for sent in obs_full]
obs_short = [[y.strip() for y in x.lower().replace('$obs:', '').strip().split(';')] for x in obs_full]
corpus = [x.replace('.', '').replace(',', '').replace('/', '') for y in obs_short for x in y if x]

In [68]:
cnt = 0
unidentified = []
for line in corpus:
    tokens = set(line.strip().split())
    tokens = {ps.stem(x) for x in tokens}
    match = False
    for identified in all_categories_stemmed:
        if len(tokens & set(identified.strip().split())) / len(tokens) > .5:
            match = True
            break
    if not match:
        unidentified.append(line)

In [71]:
def filter_(req_words, corpus):
    for req_words_set in req_words:
        assert type(req_words_set) == type(set())
    stemmed_req_words = [{ps.stem(wd) for wd in req_words_set} for req_words_set in req_words]
    
    caught = []
    not_caught = []
    for line in corpus:
        stemmed = set([ps.stem(x) for x in line.strip().split()])
        distinct = False
        for stemmed_set in stemmed_req_words:
            if stemmed_set & stemmed == set():
                distinct = True
        if not distinct:
            caught.append(line)
        else:
            not_caught.append(line)
    return caught, not_caught

difficulty_text, leftover = filter_([
    {'text', 'passage'},
    {'easy', 'challenging', 'difficult', 'level', 'hard'}
], unidentified)

accuracy_text, leftover = filter_([
    {'accurate', 'accuracy', 'miscue', 'misread'}
], leftover)

fluency_text, leftover = filter_([
    {'fluency', 'fluent', 'fluently', 'disfluent', 'intonation', 'decode', 'word-by-word', 'choppy'}
], leftover)

pronunciation_text, leftover = filter_([
    {'pronounce', 'pronunciation', 'enunciate', 'mispronounce', 'mispronunciation', 'attack'}
], leftover)

phrasing_text, leftover = filter_([
    {'phrasing'}
], leftover)

meaning_text, leftover = filter_([
    {'understanding', 'meaning', 'comprehension'}
], leftover)

speed_text, leftover = filter_([
    {'fast', 'quick', 'slow', 'slowly', 'rush', 'rate', 'quickly', 'pace'}
], leftover)

subst_omit_text, leftover = filter_([
    {'substitution', 'omission', 'omit', 'insert'}
], leftover)

punctuation_text, leftover = filter_([
    {'punctuation'}
], leftover)

self_correct_text, leftover = filter_([
    {'self-correct', 'correct', 'self'}
], leftover)

expression_text, leftover = filter_([
    {'expression', 'monotone'}
], leftover)

volume_text, leftover = filter_([
    {'quiet', 'volume'}
], leftover)

vocab_text, leftover = filter_([
    {'vocabulary'}
], leftover)

qualitative_text, leftover = filter_([
    {'good', 'solid', 'strong'}
], leftover)

In [77]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
def preprocess(line):
    return ' '.join([ps.stem(x) for x in line.strip().split()])
tfidf_vectorizer = TfidfVectorizer(preprocessor=preprocess)
tfidf = tfidf_vectorizer.fit_transform(leftover)
kmeans = KMeans(n_clusters=10).fit(tfidf)
line_to_cluster = {line: kmeans.predict(tfidf_vectorizer.transform([line]))[0] for line in leftover}
for center in set(line_to_cluster.values()):
    print(center)
    lines_with_center = [x for x in line_to_cluster if line_to_cluster[x] == center]
    for line in lines_with_center:
        print('  -', line)

0
  - noticing first letter then guessing
  - poor signal
  - needs further instruction in explicit word attackmultisyllabic word skills
  - $obs
  - careful reader
  - provide instruction in medial vowel patterns r-controlled vowel
  - occasionally runs over periods
  - consonant and vowel sounds were often incorrect
  - second language learner
  - lost place in text
  - trouble with long vowel sounds (cement plywood recreation)
  - hoarse voice quality
  - elides unstressed syllable (can provide fish)
  - difficulty with long vowel sounds and vowel teams
  - observes some periods
1
  - difficulty with the prefix un- in unusual
  - difficulty with the phoneme ph-
  - difficulty articulating r sound
  - some disfluency with stopping at periods
  - difficulty with initial sounds
  - has difficulty reading possessives
2
  - leaves off -er ending on angler
  - student removing ending of words '-ed'
  - student pausing in between sentences inconsistently
  - [how did student even get this 